# Overview

This notebook reads in all necessary datasources for the project and does some preliminary processing (clipping to the AOI, merging data layers).

# Prerequisites

## Necessary data files

| Data | Source | Notes |
| :- | -: | :-: |
| TIGER roads | https://uw-mad.maps.arcgis.com/home/item.html?id=f42ecc08a3634182b8678514af35fac3 | Primary, secondary, and local
| USFS roads | https://data.fs.usda.gov/geodata/edw/datasets.php?xmlKeyword=roads | Downloaded as geodatabase
| AOI - Mt Hood and Columbia River Gorge rec area | https://uw-mad.maps.arcgis.com/home/item.html?id=c0c7bdd467b94de2906e67f39b7cf529 | Exported as CAD files
| Visitor centers | https://data-usfs.hub.arcgis.com/datasets/usfs::recreation-opportunities-feature-layer/about/ & https://data-usfs.hub.arcgis.com/datasets/usfs::recreation-sites-feature-layer/about/| 
| Recreation data | https://uw-mad.maps.arcgis.com/home/item.html?id=c0c7bdd467b94de2906e67f39b7cf529 | Trailheads, campsites, picnic areas, ski use, paddling
| Recreation data | https://data-usfs.hub.arcgis.com/datasets/usfs::recreation-opportunities-feature-layer/about/ | Campsites, picnic areas 
| Recreation data | https://apps.nationalmap.gov/downloader/ | Trails
| Recreation data | https://github.com/openbeta/climbing-data | Climbing
| Race/ethnicity | https://www.arcgis.com/home/item.html?id=e3a7d2d3e5834b7eb6b1c2943141ced6 | 
| Education | https://data2.nhgis.org/main | Table B15003
| Income | https://data2.nhgis.org/main | Table C17002

All files were downloaded and stored in the /data/ folder of the project directory.

# Setup

In [1]:
# Import libraries
import arcpy, os, pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
# Set environment settings
arcpy.env.overwriteOutput = True

In [3]:
# Create file geodatabase to hold output
gdb_path = r"C:\Users\kathr\Documents\outdoor-alliance\mt-hood\mt-hood-analysis\mh_output.gdb"
if not arcpy.Exists(gdb_path):
    arcpy.management.CreateFileGDB(r"C:\Users\kathr\Documents\outdoor-alliance\mt-hood\mt-hood-analysis", "mh_output.gdb")
    
# Set workspace path
arcpy.env.workspace = gdb_path

# Read in Data

In [4]:
# Set path to folder where all data files are stored
data_folder = r"C:\Users\kathr\Documents\outdoor-alliance\mt-hood\data"

## Convert data in CAD format to GDB

In [6]:
# Recreation and AOI data are in CAD format
# They have been downloaded from OA ArcOnline portal and converted to CAD files already. CAD files are stored in the /data/ folder

# https://pro.arcgis.com/en/pro-app/latest/tool-reference/conversion/cad-to-geodatabase.htm
cad_data = os.path.join(data_folder, "cad-datasets")

In [7]:
reference_scale = 1000 # default value from geoprocessing tool

# For each CAD file in the directory, convert it to a geodatabase format
for path, dirs, files in os.walk(cad_data):
    for file in files:
        if file.endswith("dwg"): # CAD file
            input_cad_dataset = os.path.join(cad_data, file)
            # Set out_file_name to the name of the file with extension trimmed
            out_file_name = os.path.splitext(file)[0]
            # Call CADToGeodatabase and store in output gdb
            arcpy.conversion.CADToGeodatabase(input_cad_dataset, gdb_path, out_file_name, reference_scale)

## Roads

TIGER road data was manually added to the geodatabase by dragging and dropping the layers from the original ArcGIS Project (downloaded from ArcGIS Online using link in Prerequisites section) to the current project.

# Merge current and proposed AOI boundaries into one layer

In [5]:
# Define current layer names
cur_scenic_area = r"Current_Scenic_Area\Columbia_River_Gorge_National_Scenic_Area"
cur_wild_area = r"Current_Wildnerness_Area\Wilderness_Area__Designated__copy_"
cur_rec_area = r"Current_Rec_Area\National_Recreation_Area__Designated_2009__copy_"
prop_wild_area = r"Prop_Wilderness_Addition\Proposed_Wilderness_Addition"
prop_rec_area = r"Prop_Rec_Area_Addition\Proposed_National_Recreation_Area_Expansion" # manually split using Split tool to get 2 polygons - one overlaps with wilderness and one is new protected area

# Define layer name to hold merged layers
mh_aoi = "mh_aoi"

In [18]:
# Create field mapping object
field_mappings = arcpy.FieldMappings()

# Add input field for area name into new output field
map_name = arcpy.FieldMap()
map_name.addInputField(cur_scenic_area, "Layer")
map_name.addInputField(cur_wild_area, "Layer")
map_name.addInputField(cur_rec_area, "AREATYPE")
map_name.addInputField(prop_wild_area, "DocName")
map_name.addInputField(prop_rec_area, "Layer")

# Set name of new output field for site name
area_name = map_name.outputField
area_name.name = "Area_Name"
map_name.outputField = area_name

# Add output fields to field mappings object
field_mappings.addFieldMap(map_name)

In [19]:
# Merge all current/proposed AOI boundaries together
arcpy.management.Merge([cur_scenic_area, cur_wild_area, cur_rec_area, prop_wild_area, prop_rec_area], 
                       mh_aoi, 
                       field_mappings,
                      add_source = "ADD_SOURCE_INFO")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_aoi'>

# Buffer around AOI

In [19]:
# Create a 90 mile buffer around the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/buffer.htm

mh_expansion_buffer = "mh_expansion_buffer_90mi"
buffer_dist = "90 Miles"
dissolve_option = "ALL"

In [14]:
# Run the buffer tool
arcpy.analysis.Buffer(in_features = mh_aoi,
                      out_feature_class = mh_expansion_buffer,
                      buffer_distance_or_field = buffer_dist,
                      dissolve_option = dissolve_option)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_expansion_buffer_90mi'>

# Clip & Process Data Layers

## Roads

TIGER roads are manually clipped due to the drag/drop way of importing. Each layer (primary, secondary, and local) was clipped to the 90 mile buffer around the AOI.


In [17]:
# Clip USFS Roads to buffer around AOI
usfs_roads = os.path.join(data_folder, "S_USA.RoadCore_FS.gdb\RoadCore_FS")
usfs_roads_clipped = "usfs_roads_clipped"

In [19]:
arcpy.analysis.Clip(usfs_roads, mh_expansion_buffer, usfs_roads_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\usfs_roads_clipped'>

In [80]:
# Merge all TIGER roads
# Because these layers come from the same data source, they have the same column names and don't need additional mapping
primary_roads_clipped = "primary_roads_90mi"
secondary_roads_clipped = "secondary_roads_90mi"
local_roads_clipped = "local_roads_90mi"

In [16]:
# Create feature dataset to hold the new TIGER roads layer
sr = arcpy.Describe(primary_roads_clipped).spatialReference # get coordinate reference system
out_name = "mh_roads"
arcpy.management.CreateFeatureDataset(gdb_path, out_name, sr)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_roads'>

In [23]:
# Create feature class to hold the merge of TIGER roads and USFS roads
all_roads_merge = os.path.join(gdb_path, r"mh_roads\all_roads_merge")

In [24]:
# Create field mapping object
field_mappings = arcpy.FieldMappings()

# Add input field for road name into new output field
map_name = arcpy.FieldMap()
map_name.addInputField(primary_roads_clipped, "Name")
map_name.addInputField(secondary_roads_clipped, "Name")
map_name.addInputField(local_roads_clipped, "Name")
map_name.addInputField(usfs_roads_clipped, "NAME")

# Set name of new output field for site name
road_name = map_name.outputField
road_name.name = "Road_Name"
map_name.outputField = road_name

# Add input field for road class into new output field
map_class = arcpy.FieldMap()
map_class.addInputField(primary_roads_clipped, "MTFCC")
map_class.addInputField(secondary_roads_clipped, "MTFCC")
map_class.addInputField(local_roads_clipped, "MTFCC")
map_class.addInputField(usfs_roads_clipped, "SYMBOL_CODE")


# Set name of new output field for site type
road_class = map_class.outputField
road_class.name = "Road_Class"
map_class.outputField = road_class

# Add output fields to field mappings object
field_mappings.addFieldMap(map_name)
field_mappings.addFieldMap(map_class)

In [25]:
# Merge TIGER and USFS roads together
arcpy.management.Merge([primary_roads_clipped, secondary_roads_clipped, local_roads_clipped, usfs_roads_clipped], 
                       all_roads_merge, 
                       field_mappings,
                      add_source = "ADD_SOURCE_INFO")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_roads\\all_roads_merge'>

## Visitors Centers

In [82]:
# Recreation Opportunities: clip to buffer around AOI
rec_ops = os.path.join(data_folder, "Recreation_Opportunities_(Feature_Layer)\Recreation_Opportunities_(Feature_Layer).shp")
rec_ops_clipped = "rec_ops_clipped"

In [28]:
arcpy.analysis.Clip(rec_ops, mh_expansion_buffer, rec_ops_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\rec_ops_clipped'>

In [6]:
# Recreation Sites: clip to buffer around AOI
rec_sites = os.path.join(data_folder, "Recreation_Sites_(Feature_Layer)\Recreation_Sites_(Feature_Layer).shp")
rec_sites_clipped = "rec_sites_clipped"

In [30]:
arcpy.analysis.Clip(rec_sites, mh_expansion_buffer, rec_sites_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\rec_sites_clipped'>

In [31]:
# Recreation Sites: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_sites_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = mh_aoi

aoi_rec_sites = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [32]:
# Recreation Sites: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_sites
selection_type = "SUBSET_SELECTION"
where_clause = "SITE_SUBTY = 'INFO SITE/FEE STATION'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_sites_clipped'>

In [33]:
# Recreation Opportunities: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_ops_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = mh_aoi

aoi_rec_ops = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [34]:
# Recreation Opportunities: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_ops
selection_type = "SUBSET_SELECTION"
where_clause = "MARKERACTI = 'Visitor Centers' Or MARKERACTI = 'Visitor Programs'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_ops_clipped'>

In [35]:
# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
mh_rec_sites = "mh_rec_sites"
mh_rec_ops = "mh_rec_ops"

arcpy.management.CopyFeatures(aoi_rec_sites, mh_rec_sites)
arcpy.management.CopyFeatures(aoi_rec_ops, mh_rec_ops)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_rec_ops'>

In [37]:
# Merge all visitor centers together
visitor_centers = "visitor_centers"

# create field mapping object
field_mappings = arcpy.FieldMappings()

# Add input field for site name into new output field
map_site_name = arcpy.FieldMap()
map_site_name.addInputField(mh_rec_sites, "SITE_NAME")
map_site_name.addInputField(mh_rec_ops, "RECAREANAM")

# Set name of new output field for site name
site_name = map_site_name.outputField
site_name.name = "Site_Name"
map_site_name.outputField = site_name

# Add input field for site type into new output field
map_site_type = arcpy.FieldMap()
map_site_type.addInputField(mh_rec_sites, "SITE_SUBTY")
map_site_type.addInputField(mh_rec_ops, "MARKERACTI")

# Set name of new output field for site type
site_type = map_site_type.outputField
site_type.name = "Site_Type"
map_site_type.outputField = site_type

# add output fields to field mappings object
field_mappings.addFieldMap(map_site_name)
field_mappings.addFieldMap(map_site_type)

In [38]:
arcpy.management.Merge([mh_rec_ops, mh_rec_sites], visitor_centers, field_mappings)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\visitor_centers'>

## Trails

In [16]:
# USGS trails data

# Oregon
or_transportation = os.path.join(data_folder, r"TRAN_Oregon_State_GDB\TRAN_Oregon_State_GDB.gdb\Transportation\Trans_TrailSegment")
or_trails = "or_trails"
arcpy.management.CopyFeatures(or_transportation, or_trails)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\or_trails'>

In [17]:
# Washington
wa_transportation = os.path.join(data_folder, r"TRAN_Washington_State_GDB\TRAN_Washington_State_GDB.gdb\Transportation\Trans_TrailSegment")
wa_trails = "wa_trails"
arcpy.management.CopyFeatures(wa_transportation, wa_trails)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\wa_trails'>

In [10]:
# Merge into one trails dataset
wa_or_trails = "wa_or_trails"
arcpy.management.Merge([wa_trails, or_trails], wa_or_trails)

In [11]:
# Clip to AOI
trails_clipped = "trails_clipped"
arcpy.analysis.Clip(wa_or_trails, mh_aoi, trails_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\trails_clipped'>

In [13]:
# Recalculate length after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "trails_clipped",
                                             geometry_property = [["Length_Miles", "LENGTH_GEODESIC"]],
                                             length_unit = "MILES_INT")

<Result 'trails_clipped'>

## Whitewater paddling

In [38]:
# Clip
ww = r"Whitewater_Paddling\Whitewater_Paddling"
arcpy.analysis.Clip(ww, mh_aoi, "mh_whitewater") 
# Recalculate length
arcpy.management.CalculateGeometryAttributes(in_features = "mh_whitewater",
                                             geometry_property = [["Length_Miles", "LENGTH_GEODESIC"]],
                                             length_unit = "MILES_INT")

<Result 'mh_whitewater'>

In [41]:
# River Access: filter to within 100 m of the AOI
river_access = r"River_Access\River_Access"
arcpy.analysis.Clip(river_access, mh_aoi, "mh_river_access")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_river_access'>

## Rock climbing

In [10]:
# Read in OpenBeta data
or_areas_df = pd.read_json(os.path.join(data_folder, r"openbeta-routes-westcoast\or-areas.jsonlines"), lines=True)
or_routes_df = pd.read_json(os.path.join(data_folder, r"openbeta-routes-westcoast\or-routes.jsonlines"), lines=True)
wa_areas_df = pd.read_json(os.path.join(data_folder, r"openbeta-routes-westcoast\wa-areas.jsonlines"), lines=True)
wa_routes_df = pd.read_json(os.path.join(data_folder, r"openbeta-routes-westcoast\wa-routes.jsonlines"), lines=True)

In [11]:
or_areas_df.head()

,area_name,description,location,path,us_state,url,lnglat,metadata
0,Shepard's Tower,[A unique tower near the Metolius river.],[Take HW20 to Camp Sherman turn 2 mi west of B...,Central Oregon|Shepard's Tower,Oregon,https://www.mountainproject.com/area/105843033...,"[-121.6113, 44.64581]",{'lnglat_from_parent': False}
1,Prineville Reservoir State Park,[This area is mostly state park with some BLM....,[Once you are in Prineville turn south onto Co...,Central Oregon|Prineville Reservoir State Park,Oregon,https://www.mountainproject.com/area/121477640...,"[-120.73117, 44.13346]",{'lnglat_from_parent': False}
2,Power Lines,[A small but worthwhile climbing area to check...,"[Located about 15 minutes from Bend, drive out...",Central Oregon|Power Lines,Oregon,https://www.mountainproject.com/area/106867426...,"[-121.46742, 44.04393]",{'lnglat_from_parent': False}
3,Meadow Camp,[This is a short south and east facing basalt ...,[Head out of Bend on Cascade Lakes Highway. Ta...,Central Oregon|Meadow Camp,Oregon,https://www.mountainproject.com/area/109756477...,"[-121.37788, 44.00144]",{'lnglat_from_parent': False}
4,Youtlkut Pillars,[Youtlkut (Yu klut) Pillars is a 180-foot high...,"[From Roseburg take Hwy 138 to Glide, right on...",Southwest Oregon|Youtlkut Pillars,Oregon,https://www.mountainproject.com/area/113991218...,"[-122.6696, 43.22539]",{'lnglat_from_parent': False}


In [15]:
or_routes_df.head()

,route_name,grade,safety,type,fa,description,location,protection,metadata
0,Tetris,"{'YDS': '5.11-', 'French': '6c', 'Ewbanks': '2...",,"{'sport': True, 'tr': True}","Kevin Piarulli, May 2018","[Great position and perfect rock, Begin by ste...","[South face of the tower, left side.]","[6 bolts, chain anchor.]","{'left_right_seq': '2', 'parent_lnglat': [-121..."
1,Atlantis,"{'YDS': 'V1-2', 'Font': '5'}",,{'boulder': True},Andrew Evans,[This rock is underwater when the reservoir is...,[From the pull-off headed toward the swim beac...,[Pads],"{'left_right_seq': '999999', 'parent_lnglat': ..."
2,Unknown crack,"{'YDS': '5.9', 'French': '5c', 'Ewbanks': '17'...",,"{'trad': True, 'tr': True}",unknown,[I don't know the history of this climb. I ju...,"[Obvious crack left of hand crack, in the same...",[Trad or top rope],"{'left_right_seq': '4', 'parent_lnglat': [-121..."
3,Wide Jam,"{'YDS': '5.10a', 'French': '6a', 'Ewbanks': '1...",,"{'trad': True, 'tr': True}",unknown,[Nice thin hands to an awkward crux where the ...,[Obvious hand crack when you first get to the ...,[TR from bolts],"{'left_right_seq': '5', 'parent_lnglat': [-121..."
4,Twin Sister,"{'YDS': '5.9', 'French': '5c', 'Ewbanks': '17'...",,"{'trad': True, 'tr': True}","David Campbell and David Vanslyke, 1994.","[A face climb using two columns, finishing on ...",[Center left of the columns.],[trad gear or top anchor bolts.],"{'left_right_seq': '3', 'parent_lnglat': [-122..."


In [16]:
# Split longitude and latitude values into separate columns
or_areas_df["Longitude"] = or_areas_df["lnglat"].str[0]
or_areas_df["Latitude"] = or_areas_df["lnglat"].str[1]

wa_areas_df["Longitude"] = wa_areas_df["lnglat"].str[0]
wa_areas_df["Latitude"] = wa_areas_df["lnglat"].str[1]

In [18]:
# Merge the areas dataframes together
wa_or_climbing_areas = pd.concat([or_areas_df, wa_areas_df])

In [20]:
# Create spatial dataframe
wa_or_spatial = pd.DataFrame.spatial.from_xy(df = wa_or_climbing_areas,
                                               x_column = "Longitude",
                                               y_column = "Latitude",
                                               sr = 4326)

In [21]:
# Write combined climbing dataframes back into one feature class
wa_or_spatial.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "wa_or_climbing"), 
                                        sanitize_columns = False)

<da.funcInfo object at 0x000001EA9C9F4330> returned NULL without setting an error


'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\wa_or_climbing'

In [29]:
# Clip to AOI
wa_or_climbing = "wa_or_climbing"
climbing_clipped = "climbing_clipped"
arcpy.analysis.Clip(wa_or_climbing, mh_aoi, climbing_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\climbing_clipped'>

## Picnic

In [9]:
# Recreation Opportunities: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_ops_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = mh_aoi

aoi_rec_ops = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [10]:
# Recreation Opportunities: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_ops
selection_type = "SUBSET_SELECTION"
where_clause = "MARKERACTI = 'Picnicking'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_ops_clipped_Layer1'>

In [11]:
# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
mh_picnic ="mh_picnic"

arcpy.management.CopyFeatures(aoi_rec_ops, mh_picnic)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_picnic'>

## Camping

In [83]:
# Recreation Opportunities: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_ops_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "10 Meters"
selection_type = "NEW_SELECTION"
selecting_features = mh_aoi

aoi_rec_ops = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [84]:
# Recreation Opportunities: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_ops
selection_type = "SUBSET_SELECTION"
where_clause = "MARKERAC_1 = 'Camping & Cabins'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_ops_clipped_Layer1'>

In [85]:
# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
mh_camping = "mh_camping"

arcpy.management.CopyFeatures(aoi_rec_ops, mh_camping)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_camping'>

## Census

### 2020 Redistricting Tracts (Racial Data)

In [39]:
# Clip census to buffer around AOI
census = os.path.join(data_folder, r"Census\Default.gdb\census_2020")
census_clipped = "census_clipped"

In [40]:
arcpy.analysis.Clip(census, mh_expansion_buffer, census_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\census_clipped'>

In [41]:
# Select census data for states of WA/OR
in_layer = census
selection_type = "NEW_SELECTION"
where_clause = "State_Name = 'Oregon' or State_Name = 'Washington'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

# This creates a temporary layer on the map (but not in the gdb) called "census_2020_Layer1"

<Result 'census_2020_Layer1'>

In [44]:
# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
wa_or_census = "wa_or_census"

arcpy.management.CopyFeatures("census_2020_Layer1", "wa_or_census")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\wa_or_census'>